# Histogrammar: a set of data aggregation primitives

Histogrammar is a set of data aggregation and statistical analysis primitives that integrates well with Spark, Scala and Spark SQL (oh, and Python). 

Histogrammar is an open source (Apache 2.0) project where the 2 main contributors are from Princeton University (Jim Pivarski and Alexey Svyatkovskiy - me).

More details and tutorials are avilable here: http://histogrammar.org/docs/

## Installation instructions

Histogrammar is available on Maven Central, a publicly accessible Java/Scala repository with dependency management.

### Apache Spark

To use Histogrammar in the Spark shell, you don’t have to download anything. Just start Spark with (same as we did with Databrick's spark-csv)

```bash
spark-shell --packages "org.diana-hep:histogrammar_2.11:1.0.3"
```

and call

```scala
import org.dianahep.histogrammar._
```

on the Spark prompt. For plotting with Bokeh, include `org.diana-hep:histogrammar-bokeh_2.11:1.0.3` and for interaction with Spark-SQL, include `org.diana-hep:histogrammar-sparksql_2.11:1.0.3`.

Use `_2.11` for compatibility with Spark 2.0.0 (Scala 2.11) and `_2.10` for compatibility with Spark 1.x (Scala 2.10).
Note: due to a dependency bug, Bokeh is incompatible with Spark 2.x (Scala 2.11).

### Java/Scala with Maven

To compile Histogrammar into a project with the Maven build tool, add

```
<dependency>
  <groupId>org.diana-hep</groupId>
  <artifactId>histogrammar_2.11</artifactId>
  <version>1.0.3</version>
</dependency>
```

to your `<dependencies>` section. Use `_2.11` for compatibility with Scala 2.11 and `_2.10` for compatibility with Scala 2.10.

### Scala with sbt

To use Histogrammar in sbt console or to compile it into a project with the sbt build tool, add

```
libraryDependencies += "org.diana-hep" %% "histogrammar" % "1.0.3"
```
to your build.sbt file. The double-percent gets the appropriate version of Histogrammar for your version of Scala.

More specifics on the installation can be found here:

http://histogrammar.org/docs/install

In [ ]:
More specifics on the installation can be found here:

http://histogrammar.org/docs/install